In [8]:
import pandas as pd
from tqdm.notebook import tqdm
import os,sys

if os.getcwd()[-4:] == "code":
    os.chdir('../')

import module
from datetime import datetime, timedelta

In [2]:
iculst = ["outputevents", "procedureevents", "icustays","chartevents"]

for i in iculst:
    globals()[i] = pd.read_parquet("mimic-iv-2.2-parquet/icu/%s.parquet"%i)

In [3]:
hosplst = ["admissions", "procedures_icd", "omr"]

for i in hosplst:
    globals()[i] = pd.read_parquet("mimic-iv-2.2-parquet/hosp/%s.parquet"%i)

In [4]:
# Urine
# 2.1 Urine
#Urine
Urine_icu = outputevents[outputevents['itemid'].isin([
    226557, 226558, #Ureteral Stent, 요관스텐트
    226559, #Foley, 도뇨관(소변줄)
    226560, 226561, 226563, 226564, 226565, 226566, 226567, 226584, 226627, 226631, 226632
])]
Urine_icu_guirrigant_input = outputevents[outputevents['itemid'].isin([227488])]
Urine_icu_guirrigant_output = outputevents[outputevents['itemid'].isin([227489])]
guirrigant = pd.merge(Urine_icu_guirrigant_input,Urine_icu_guirrigant_output[['charttime','value']],on='charttime')
guirrigant['value'] =  guirrigant['value_y'] - guirrigant['value_x']
guirrigant = guirrigant[guirrigant['value'] >= 0]
guirrigant = guirrigant.drop(columns=['value_x','value_y','itemid','valueuom'])
Urine_icu = pd.concat([Urine_icu,guirrigant]).sort_values(['subject_id','charttime'])
Urine_icu = pd.merge(Urine_icu,icustays[['stay_id','intime','outtime']].drop_duplicates(),on='stay_id',how='inner')
Urine_icu = Urine_icu.drop(columns=['itemid','valueuom','storetime'])
Urine_icu = Urine_icu[Urine_icu ['value'] >= 0]
date_columns = ['charttime','intime','outtime']
Urine_icu[date_columns] = Urine_icu[date_columns].apply(pd.to_datetime).drop_duplicates()
Urine_icu = Urine_icu[(Urine_icu['charttime'] > Urine_icu['intime']) & (Urine_icu['charttime'] < Urine_icu['outtime'])]
Urine_icu = Urine_icu.rename(columns={'value': 'Urine'})

In [5]:
# 2.2 RRT_icu & KT & LOS
#RRT_icu
RRT_icu =  (procedureevents[procedureevents['itemid'].isin([225441, 225802, 225805])]
           .assign(starttime=lambda x: pd.to_datetime(x['starttime']),
                   endtime=lambda x: pd.to_datetime(x['endtime']))
           .merge(icustays[['stay_id', 'intime', 'outtime']], on='stay_id', how='inner')
           .sort_values(['subject_id', 'starttime'])
           .reset_index(drop=True))
RRT_icu['itemid'].value_counts()
Urine_icu['RRT'] = 0
Urine_icu = Urine_icu.groupby('stay_id').apply(lambda x: module.filter_RRT_icu(x, RRT_icu[RRT_icu['stay_id'] == x.name])).reset_index(drop=True)
#KT
KT = (procedures_icd[procedures_icd['icd_code'].isin(['5569'])]
            .assign(chartdate=lambda x: pd.to_datetime(x['chartdate']))
            .merge(admissions[['hadm_id', 'admittime', 'dischtime']], on='hadm_id', how='left')
            .sort_values(['subject_id', 'chartdate'])
            .reset_index(drop=True))
KT['icd_code'].value_counts()
Urine_icu = Urine_icu.groupby('subject_id').apply(lambda x: module.filter_KT(x, KT[KT['subject_id'] == x.name])).reset_index(drop=True)
#LOS
LOS = icustays[((icustays['los'] > 1) & (icustays['los'] <= 14))]
Urine_icu = pd.merge(Urine_icu,LOS[['stay_id','los']],how='inner', on='stay_id')

In [6]:
# 2.3 Weight
#Weight
Weight_icu_kg = chartevents[chartevents['itemid'].isin([224639,226512])]
Weight_icu_lbs = chartevents[chartevents['itemid'].isin([226531])]
Weight_icu_lbs['valuenum'] = Weight_icu_lbs['valuenum'] / (1 / 0.453592)
Weight_icu = pd.concat([Weight_icu_kg,Weight_icu_lbs])
Weight_icu = Weight_icu[(Weight_icu.valuenum <= 250) & (Weight_icu.valuenum > 0)]
Weight_icu['valuenum'] = round(Weight_icu['valuenum'],1)
Weight_icu['charttime'] = pd.to_datetime(Weight_icu['charttime'])
Weight_icu = Weight_icu.rename(columns={'valuenum': 'Weight'})
omr_weight_kg = omr[omr['result_name'].isin(['Weight'])]
omr_weight_lbs = omr[omr['result_name'].isin(['Weight (Lbs)'])]
omr_weight_kg['result_value'] = omr_weight_kg['result_value'].astype(float)
omr_weight_lbs['result_value'] = omr_weight_lbs['result_value'].astype(float)
omr_weight_lbs['result_value'] = omr_weight_lbs['result_value'] / (1 / 0.453592)
omr_weight = pd.concat([omr_weight_kg,omr_weight_lbs])
omr_weight.chartdate = pd.to_datetime(omr_weight.chartdate + ' 23:59:59')
omr_weight['charttime'] = omr_weight['chartdate']
omr_weight['Weight'] = omr_weight['result_value']
omr_weight = omr_weight[omr_weight.Weight <= 250]
omr_weight = omr_weight[omr_weight.Weight > 0]
omr_weight['Weight'] = round(omr_weight['Weight'],1)
omr_weight = omr_weight.sort_values(['subject_id','charttime']).reset_index(drop=True)
omr_weight = omr_weight.drop_duplicates(subset=['subject_id','charttime'],keep='last')
Weight_check = pd.concat([omr_weight['subject_id'].drop_duplicates(),Urine_icu['subject_id'].drop_duplicates()]).drop_duplicates()
Urine_icu = pd.merge(Urine_icu,Weight_check,on='subject_id',how='inner')
omr_weight = omr_weight[['subject_id','charttime','Weight']]
Weight_icu = Weight_icu[['subject_id','charttime','Weight']]
Weight_pool = pd.concat([omr_weight,Weight_icu]).sort_values(['charttime'])
Urine_icu = Urine_icu.groupby('subject_id',group_keys=False).apply(lambda x : module.Weight(x, Weight_pool[Weight_pool['subject_id'] == x.name])).reset_index(drop=True)
Urine_icu.to_parquet('Urine_icu_Weight.parquet')

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_1222/1273243504.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Weight_icu_lbs['valuenum'] = Weight_icu_lbs['valuenum'] / (1 / 0.453592)
/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_1222/1273243504.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_weight_kg['result_value'] = omr_weight_kg['result_value'].astype(float)
/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_1222/1273243504.py:14: SettingWithCopyWarning: 
A

In [9]:
# 2.4 Urine
# 체중 없음, 체중 1kg 이하 , 측정 간격 n 시간 이상, 측정 간격 n 시간 이하,
def Urine(df):
    df = df.sort_values(by=['stay_id', 'charttime'])
    df['charttime_diff'] = df.groupby('stay_id')['charttime'].diff().fillna(timedelta(seconds=0))
    df = df.assign(**{'6h-12h': 0, '12h': 0, '24h': 0, 'anuria_12h': 0, 'Urine_stage': 0, 'Urine_output_rate': 0})
    anuria_threshold = 50.0
    def process_group(target):
        target = target.reset_index(drop=True)
        target['Urine_output_rate'] = target['Urine'] / (target['charttime_diff'].dt.total_seconds() / 3600.0) / target['Weight']
        target['cum_value'] = target['Urine'][::-1].cumsum()
        target['cum_time_diff'] = target['charttime_diff'][::-1].cumsum().dt.total_seconds() / 3600.0
        for i in range(1, len(target)):
            group = target.iloc[1:i+1]
            group['cum_value'] = group['cum_value'] - group['cum_value'].iloc[-1]
            group['cum_time_diff'] = group['cum_time_diff'] - group['cum_time_diff'].iloc[-1]
            for threshold_hours_min, threshold_hours_max, rate_threshold, column_name, stage in [
                (6, 12, 0.5, '6h-12h', 1),
                (12, float('inf'), 0.5, '12h', 2),
                (24, float('inf'), 0.3, '24h', 3)]:
                condition = (group['cum_time_diff'] >= threshold_hours_min) & (group['cum_time_diff'] < threshold_hours_max)
                filtered_group = group.loc[condition]
                if not filtered_group.empty:
                    urine_output_rate = filtered_group['cum_value'] / filtered_group['cum_time_diff'] / target['Weight'].iloc[i]
                    if (column_name in ['6h-12h', '12h', '24h']) and (urine_output_rate.iloc[-1] < rate_threshold):
                        target.loc[i, column_name] = 1
                        target.loc[i, 'Urine_stage'] = stage
                    if (column_name == '12h') and (filtered_group['cum_value'].iloc[-1] < anuria_threshold):
                        target.loc[i, 'anuria_12h'] = 1
                        target.loc[i, 'Urine_stage'] = 3
        return target
    tqdm.pandas(desc="Processing groups")
    df = df.groupby('stay_id', group_keys=False).progress_apply(process_group).reset_index(drop=True)
    df = df[(~df['Urine_output_rate'].isna()) & (df['Urine_output_rate'] != float('inf'))]
    df = df.groupby('stay_id', group_keys=False).apply(lambda x : module.gap(x,'Urine_output_rate')).reset_index(drop=True)
    df['Urine_charttime_diff'] = round(df['charttime_diff'].dt.total_seconds() / 3600,1)
    df.loc[df['Urine_charttime_diff'] >= 12, ['Urine_stage','anuria_12h']] = 3
    return df
Urine_icu = Urine(Urine_icu)
Urine_icu.to_parquet("Urine_icu.parquet")
Urine_icu = pd.read_parquet('Urine_icu.parquet')
date_columns = ['charttime', 'intime', 'outtime']
Urine_icu[date_columns] = Urine_icu[date_columns].apply(pd.to_datetime)
Urine_icu['Urine_stage'].value_counts()

Processing groups:   0%|          | 0/53171 [00:00<?, ?it/s]

In [11]:
Urine_icu.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime',
       'Urine', 'intime', 'outtime', 'RRT', 'los', 'Weight', 'charttime_diff',
       '6h-12h', '12h', '24h', 'anuria_12h', 'Urine_stage',
       'Urine_output_rate', 'cum_value', 'cum_time_diff',
       'Urine_output_rate_diff', 'Urine_charttime_diff'],
      dtype='object')

In [14]:
Urine_icu.Urine_stage.value_counts()

Urine_stage
0    1765725
2     275990
1     150853
3      86830
Name: count, dtype: int64

In [15]:
Urine_stage

NameError: name 'Urine_stage' is not defined